In [ ]:
from IPython.core.display import display, HTML
from matplotlib.pyplot import rc
display(HTML("<style>.container { width:100% !important; }</style>"))
font = {'family' : 'monospace',
        'weight' : 'bold',
        'size'   : '12'}
rc('font', **font)  # pass in the font dict as kwargs

In [ ]:
from qiskit import IBMQ
mytoken = '105ccb14a9f37cd5ce743b6e47f48c0da24790f0432159de03b522fa68d16848eb741951dcbe5f2aa0af2e0ce3abd5be008d9e37042da2b95e83dbef772850f0'
IBMQ.save_account(mytoken, overwrite=True)

In [ ]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.aer import QasmSimulator
from matplotlib import pyplot as plt
import qiskit.providers.aer.noise as noise
# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()
for backend in provider.backends():
    print(backend)

In [ ]:
from qiskit.extensions import UnitaryGate
sqrtX = UnitaryGate([[1/np.sqrt(2), -1/np.sqrt(2)*1j], [-1/np.sqrt(2)*1j, 1/np.sqrt(2)]], label='X^1/2')
sqrtY = UnitaryGate([[1/np.sqrt(2), -1/np.sqrt(2)], [1/np.sqrt(2), 1/np.sqrt(2)]], label='Y^1/2')
sqrtW = UnitaryGate([[1/np.sqrt(2), -1/np.sqrt(2)*np.sqrt(1j)], [1/np.sqrt(2)*np.sqrt(-1j), 1/np.sqrt(2)]], label='W^1/2')

In [ ]:
# test that add gate works
qc = QuantumCircuit(3)
qc.draw()
qc.append(sqrtX, [0])
qc.append(sqrtY, [1])
qc.append(sqrtW, [2])
qc.draw()

In [ ]:
import random
def random_gate(num_qubits, operands):
    """
    Args:
    - operands - gates to choose from
    Out:
     - Random unitary gate from operands
    """
    choices = random.choices(population = operands, k = num_qubits)
    if len(choices) == 1:
        return choices[0]
    return choices

In [ ]:
std_operands = [sqrtX, sqrtY, sqrtW]
rand_gate_test = random_gate(num_qubits = 1, operands= std_operands)
rand_gate_test.to_matrix()

In [ ]:
def gen_rqc(num_qubits, depth, operands = std_operands, seed = 2022):
    reg = [i for i in range(num_qubits)]
    random.seed(seed)
    qc = QuantumCircuit(num_qubits)
    qc.h([i for i in range(qc.num_qubits)])
    choices = [None]*num_qubits
    for i in range(depth):
        # cz gate layer
        for j in range(num_qubits):
            choice = random_gate(num_qubits=1, operands = operands)
            while choice == choices[j]:
                choice = random_gate(num_qubits=1, operands = operands)
            choices[j] = choice # store to make sure the same gate does not get chosen two times consequtively
            qc.unitary(choice, [j], label = choice.label)
        # random gate layer
        if i%2 == 0:
            igate = np.arange(0,num_qubits-1, 2)
            for j in igate: 
                qc.cz(j, j+1)
        else:
            igate = np.arange(1,num_qubits-1, 2)
            for j in igate:
                qc.cz(j, j+1)
        qc.barrier()
                   
    return qc


In [ ]:
import pylatexenc

In [ ]:
qc5 = gen_rqc(num_qubits=15, depth = 40)
qc5.save_statevector()
print(qc5)

In [ ]:
def num_2gates(num_qubits, depth):
    even = (num_qubits%2 == 0)
    if even:
        g2 = (num_qubits-1)*depth//2 +1 # every second layer there is 1 cz gate less, the +1 is because // rounds down to closest integer
    else:
        g2 = (num_qubits//2)*depth # // rounds down to closes integer
    return g2
num_2gates(14, 5)

In [ ]:
circ_test = gen_rqc(num_qubits=14, depth = 5)
print(circ_test)

In [ ]:
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
backend = provider.get_backend('ibmq_lima')
backend_noise_model = NoiseModel.from_backend(backend)

#Create the simulators

# Create ideal simulator backend and transpile circuit
sim_ideal = AerSimulator(method='statevector')

#Design noise model
depth = 40
# Error parameters
r = 0.0005
rinit = r # error rate for initialisation of the qrc
r1 = r/10 # gate 1 error rate
r2 = r # gate 2 error rate
g1 = 15*depth # number of single qbit gates
g2 = num_2gates(num_qubits = 15, depth = depth) # number of 2-qbit gates
rmes = r # error rate for measurements

# initialise the noise model
designed_noise_model = noise.NoiseModel()
# Construct the error
bit_flip = noise.pauli_error([('X', r), ('I', 1 - r)]) # construct bit-flip error

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(r1, 1)
error_2 = noise.depolarizing_error(r2, 2)

designed_noise_model.add_all_qubit_quantum_error(bit_flip, "reset")
designed_noise_model.add_all_qubit_quantum_error(error_1, ['unitary', 'X^1/2', 'Y^1/2', 'W^1/2'])
designed_noise_model.add_all_qubit_quantum_error(error_2, ['cz'])
designed_noise_model.add_all_qubit_quantum_error(bit_flip, "measure")

qc5.measure_all()

In [ ]:
sim_noise = AerSimulator(noise_model=designed_noise_model)

# ideal simulation
res_noisefree = sim_ideal.run(qc5).result()
state = res_noisefree.get_statevector()
pU_test = state.probabilities()

# noisy simulation
qct5 = transpile(qc5, sim_noise)
res_noise = sim_noise.run(qct5, shots=int(1e3)).result()
pnoise = res_noise.get_statevector().probabilities_dict()

In [ ]:
import qiskit.quantum_info
def prob_getcounts(result, qc):
    Nshots = sum(result.get_counts(qc).values())
    print(Nshots)
    x = {}
    Nshots = sum(list(result.get_counts().values() ))
    for bitstring, count in result.get_counts().items():
        bit = bitstring.split(" ")[0]
        x[bit] = count/Nshots
    return x
# check sum of prob equals 1
p_noisefree = prob_getcounts(result = res_noisefree, qc = qc5)
p_noisy = prob_getcounts(result = res_noise, qc = qc5)

In [ ]:
def get_puexp(probdict_noisy, pU):
    norm = 0
    puexp = []
    for key, value in probdict_noisy.items():
        ind = int(key.split(" ")[0], 2) # indices of the bitstrings, have to split between qubits and classical bits
        puexp.append(pU[ind])
    puexp = np.array(puexp)
    return puexp

def compute_entropy(probs):
    # print("\n entropy function probs: ", probs)
    return - np.sum(probs*np.log(probs))


# calculate the cross entropy difference estimate
def cross_entropy_est(probdict_noisy, pU, euler_const = 0.577):
    num_qubits = len(list(probdict_noisy.keys())[0].split(" ")[0])
    print("num_qubits", num_qubits)
    #calculate the estimate of the fidelity aka cross entropy difference
    H0 = num_qubits*np.log(2) + euler_const
    H_pA_pU = 0
    sumpu = 0
    nshots = sum(list(probdict_noisy.values() ))
    #print("nshots", nshots)
    print("number of unique samples drawn", len(list(probdict_noisy.keys())), "\ntot number of possible samples", 2**(num_qubits))
    for bistring, count in probdict_noisy.items():
        ind = int(bistring.split(" ")[0], 2) # indices of the bitstrings, have to split between qubits and classical bits
        #print(ind, count)
        #print([bin(i) for i in range(2**(8))][ind], bistring.split(" ")[0]) # confirm that the right key is found
        sumpu += pU[ind]
        #print(count)
        #print("pU[ind]", pU[ind])
        H_pA_pU += (count/nshots)*np.log( pU[ind] )
    #print('sumpu', sumpu)
    alpha = H0 + H_pA_pU
    return alpha, -H_pA_pU/nshots

In [ ]:
"""testcount = {'00':100, '01':5, '10':15, '11':80}
testpU = [.45, 0.05, 0.1, .4]
#testpU1 = [.5, 0.05, 0.1, .4]
#get_puexp(testcount, testpU)
cross_entropy_est(probdict_noisy = testcount, pU = testpU )"""

In [ ]:
#print(prob_noisefree)
#cross_entropy_est(probdict_noisy = res_noisefree.get_statevector().probabilities_dict(), pU = pU_test) # probdict from statevector from noisefree simulator for comparison
#cross_entropy_est(probdict_noisy = res_noise.get_statevector().probabilities_dict(), pU = pU_test) # probdict from statevector from noisy simulator for comparison
cross_entropy_est(probdict_noisy = res_noise.get_counts(qc5), pU = pU_test ) # from getcounts

In [ ]:
def alpha_digital(r, depth, num_qubits):
    g1 = num_qubits*depth # number of single qbit gates
    g2 = num_2gates(num_qubits = num_qubits, depth = depth) # number of 2-qbit gates
    return np.exp(-r*(g1/10 + g2 + 2*num_qubits))
alpha_dig_test = alpha_digital(r=r, depth=depth, num_qubits = 9)
alpha_dig_test

In [ ]:
# calculate the cross entropy difference estimate
def compute_IPRK(probdict_noisfree, pU, k, euler_const = 0.577):
    num_qubits = len(list(probdict_noisfree.keys())[0].split(" ")[0])
    iprk = 0
    for key, value in probdict_noisfree.items():
        ind = int(key.split(" ")[0], 2) # indices of the bitstrings, have to split between qubits and classical bits
        iprk += np.abs(value* pU[ind] )**(2*k)
    return iprk
compute_IPRK(probdict_noisfree = p_noisy, pU = pU_test, k = 2)

In [ ]:
import pandas as pd
ks = np.arange(2,11,2)
df_qs = pd.DataFrame(columns= ['alpha_dig', 'puexp', 'ninstance', 'num_qubits', 'depth', 'pU','entropy','cross_entropy', 'cross_entropy_diff','nshots']+['IPR%i'%k for k in ks])
df_qs

In [ ]:
# make list of entropy depth pairs
def entropy_vs_depth(df = df_qs, r = 0.0005, num_qubits = 15, depth_range = np.arange(20,30,2), Nshots = int(2e3), ninstance = None, ks = None, plot_hist_NpU = False):
    N = 2**(num_qubits)
    
    # get backend and noise model for simulating real hardware
    backend = provider.get_backend('ibmq_lima')
    hardware_noise_model = NoiseModel.from_backend(backend)
    sim_real = QasmSimulator(method='density_matrix', noise_model=hardware_noise_model)
    qc = None
    for i in range(len(depth_range)):
        depth = depth_range[i]
        # generate the quantum circuit
        if qc is not None:
            del qc
        qc = gen_rqc(num_qubits=num_qubits, depth = depth)
        qc.save_statevector()
        
        # Create ideal simulator backend and transpile circuit
        sim_ideal = AerSimulator(method='statevector')
        
        #Design noise model

        # Error parameters
        rinit = r # error rate for initialisation of the qrc
        r1 = r/10 # gate 1 error rate
        r2 = r # gate 2 error rate
        g1 = num_qubits*depth # number of single qbit gates
        g2 = num_2gates(num_qubits = num_qubits, depth = depth) # number of 2-qbit gates
        rmes = r # error rate for measurements
        
        # Initialise the noise model
        designed_noise_model = noise.NoiseModel()
        
        # Construct the errors
        # bitflip error
        bit_flip = noise.pauli_error([('X', r), ('I', 1 - r)]) # construct bit-flip error
        
        # Depolarizing quantum errors
        error_1 = noise.depolarizing_error(r1, 1)
        error_2 = noise.depolarizing_error(r2, 2)
        depolnum = 4/(4-1)
        
        designed_noise_model.add_all_qubit_quantum_error(bit_flip, "reset")
        designed_noise_model.add_all_qubit_quantum_error(error_1, ['unitary'])
        designed_noise_model.add_all_qubit_quantum_error(error_2, ['cz'])
        designed_noise_model.add_all_qubit_quantum_error(bit_flip, "measure")
        #print(designed_noise_model)

        
        # t_qc_noisy = transpile(qc, sim_noise)
        
        # measure all qubits
        qc.measure_all()
        
        # get probabilites from ideal circuit
        result_ideal = sim_ideal.run(qc).result()
        state = result_ideal.get_statevector()
        pU = state.probabilities()
        
        # Create noisy simulator backend and transpile circuit
        sim_noise = AerSimulator(noise_model=designed_noise_model)
        qct = transpile(qc, sim_noise) # transpile for noisy basis gates
        # get noisy probabilites
        result_noisy = sim_noise.run(qct, shots=Nshots).result()
        probs_noisy = prob_getcounts(result = result_noisy, qc = qc)
        
        #compute pu(x^exp)
        puexp = get_puexp(probdict_noisy = probs_noisy, pU = pU)
        
        #compute entropy
        entropy = compute_entropy(probs = np.array(list(probs_noisy.values())) )
        
        #compute cross entropy and cross entropy difference
        crentropy_diff, crentropy = cross_entropy_est(probdict_noisy = result_noisy.get_counts(qc), pU = pU )
        #crentropy_diff0, crentropy0 = cross_entropy_est(probdict_noisy = result_ideal.get_counts(qc), pU = pU) # to check if the same answer is given for r = 0
        #print("crentropy_diff0-crentropy_diff", crentropy_diff0-crentropy_diff)
        # compute iprk
        IPR_k = [compute_IPRK(probdict_noisfree = result_noisy.get_counts(qc), pU = pU, k = k) for k in ks]
        
        #compute alpha digital eq 19 in paper
        alpha_dig = alpha_digital(r = r, depth=depth, num_qubits = num_qubits)
        #append to output        
        #print([float(IPR_k[ik]) for ik in range(len(ks))])
        tempdf = pd.DataFrame([[alpha_dig, tuple(puexp), ninstance, num_qubits, depth, tuple(pU), r, 
                                entropy, crentropy, crentropy_diff, Nshots] + [IPR_k[ik] for ik in range(len(ks))]], 
                              columns= ['alpha_dig', 'puexp', 'ninstance', 'num_qubits', 'depth', 'pU', 'r',
                                        'entropy','cross_entropy', 'cross_entropy_diff','nshots']+ ['IPR%i'%k for k in ks])
        df = pd.concat([df, tempdf])
    return df

In [ ]:
dfmaster = pd.DataFrame(columns= ['alpha_dig', 'puexp', 'ninstance', 'num_qubits', 'depth', 'pU','entropy','cross_entropy', 'cross_entropy_diff','nshots']+['IPR%i'%k for k in ks])
dfmaster

In [ ]:
#depth_range = np.arange(20,41,5)
depth_range = list(np.arange(5,20,1)) + [20,25,30,35,40]
#rs = np.array([0, 0.0005, 0.001, 0.002, 0.005, 0.01])
rs = np.array([0.02  , 0.    , 0.001 , 0.0005, 0.002 , 0.005 , 0.01  ])
Ninstances = 10
shape = rs.shape + (Ninstances,) + depth_range.shape
depth5, cross_entropy5, cross_entropy_diff5, IPRK5, entropy5 = np.zeros(shape), np.zeros(shape), np.zeros(shape), np.zeros(rs.shape + (Ninstances,)+ ks.shape +depth_range.shape), np.zeros(shape)

for j in range(len(rs)):
    r = rs[j]
    print("r", r)
    for i in range(Ninstances):
        df_qs = entropy_vs_depth(num_qubits = 15, Nshots = int(1e3), r = r, depth_range = depth_range, ks=ks, ninstance = i)
        dfmaster = pd.concat([dfmaster, df_qs])

In [ ]:
dfmaster.to_csv('/home/IPP-HGW/joag/homework/db_qcqi_exam_final_5to19_20_40_step5')

In [ ]:
dfmaster_stds = pd.DataFrame(columns= ['alpha_dig', 'ninstance', 'num_qubits', 'depth', 'r',
                                        'entropy','cross_entropy', 'cross_entropy_diff','nshots']+ ['IPR%i'%k for k in np.arange(2,11,2)])
for d in sorted(dfmaster['depth'].unique()):
    for r in sorted(dfmaster['r'].unique()):
        inds = dfmaster['r'].isin([r]) & dfmaster['depth'].isin([d])
        tdfm = dfmaster[inds].std()
        tempdf = pd.DataFrame([tdfm], columns= ['alpha_dig', 'ninstance', 'num_qubits', 'depth', 'r',
                                            'entropy','cross_entropy', 'cross_entropy_diff','nshots']+ ['IPR%i'%k for k in np.arange(2,11,2)])
        dfmaster_stds = pd.concat([dfmaster_stds, tempdf])
dfmaster_stds

In [ ]:
dfmaster_stds.to_csv('/home/IPP-HGW/joag/homework/db_qcqi_exam_std_5to19_20_40_step5')

In [ ]:
dfmaster_mean_noprobs = pd.DataFrame(columns= ['alpha_dig', 'ninstance', 'num_qubits', 'depth', 'r',
                                        'entropy','cross_entropy', 'cross_entropy_diff','nshots']+ ['IPR%i'%k for k in np.arange(2,11,2)])
for d in sorted(dfmaster['depth'].unique()):
    for r in sorted(dfmaster['r'].unique()):
        inds = dfmaster['r'].isin([r]) & dfmaster['depth'].isin([d])

        tdfm = dfmaster[inds].mean()
        tempdf = pd.DataFrame([tdfm], columns= ['alpha_dig', 'puexp', 'ninstance', 'num_qubits', 'depth', 'pU', 'r',
                                            'entropy','cross_entropy', 'cross_entropy_diff','nshots']+ ['IPR%i'%k for k in np.arange(2,11,2)])
        dfmaster_mean_noprobs = pd.concat([dfmaster_mean_noprobs, tempdf])
dfmaster_mean_noprobs

In [ ]:
dfmaster_mean_noprobs.to_csv('/home/IPP-HGW/joag/homework/db_qcqi_exam_means_noprobs_5to19_20_40_step5')

In [ ]:
"""dfmaster_mean = pd.DataFrame(columns= ['alpha_dig', 'puexp', 'ninstance', 'num_qubits', 'depth', 'pU', 'r',
                                        'entropy','cross_entropy', 'cross_entropy_diff','nshots']+ ['IPR%i'%k for k in np.arange(2,11,2)])
for d in sorted(dfmaster['depth'].unique()):
    for r in sorted(dfmaster['r'].unique()):
        inds = dfmaster['r'].isin([r]) & dfmaster['depth'].isin([d])
        pus = []
        puexps = []
        for i,row in dfmaster[['pU', 'puexp']].iterrows():
            puexps.append(np.array(list(row['puexp'])))
            pus.append(np.array(list(row['pU'])))
        tdfm = dfmaster[inds].mean()
        tdfm['pU'] = np.mean(np.array(pus), axis = 0)
        tdfm['puexp'] = np.array(puexps), axis = 0
        tempdf = pd.DataFrame([tdfm], columns= ['alpha_dig', 'puexp', 'ninstance', 'num_qubits', 'depth', 'pU', 'r',
                                            'entropy','cross_entropy', 'cross_entropy_diff','nshots']+ ['IPR%i'%k for k in np.arange(2,11,2)])
        dfmaster_mean = pd.concat([dfmaster_mean, tempdf])
dfmaster_mean"""

In [ ]:
#dfmaster_mean.to_csv('/home/IPP-HGW/joag/homework/db_qcqi_exam_mean_5to19_20_40_step5')

In [ ]:
from matplotlib import cm
from matplotlib.colors import Normalize as clnormalize
num_qubit = max(dfmaster['num_qubits'].unique())
fig, ax = plt.subplots(figsize=(10,10))
viridis = cm.get_cmap('viridis', 20)
sm = cm.ScalarMappable(cmap=viridis, norm = clnormalize(dfmaster['r'].unique()[0], dfmaster['r'].unique()[-1]))
ind_d = dfmaster_mean_noprobs['depth'].isin([15, 16, 17, 18, 19, 20, 25, 30, 35, 40])
for r in dfmaster_mean_noprobs['r'].unique():
    indr = dfmaster_mean_noprobs['r'].isin([r])
    items = dfmaster_mean_noprobs[['depth', 'cross_entropy_diff', 'alpha_dig']][indr&ind_d]
    line1 = ax.plot(items['depth'], items['cross_entropy_diff'], '-X', label = '$r_{est} = %.3f$'%r, c = viridis(r*50))
    line2 = ax.plot(items['depth'], items['alpha_dig'], '--o', label = '$r_{dig} = %.3f$'%r, c = viridis(r*50))
    ebar1 = ax.errorbar(items['depth']), items['cross_entropy_diff'],
                        yerr=items['cross_entropy_diff'],ecolor = viridis(r*50), color = viridis(r*50))
plt.colorbar(sm)
plt.legend(loc='lower right')
plt.xlabel("Depth")
plt.ylabel("Fidelity")

In [ ]:
def pralpha_z(z, alpha):
    """
    z = log(Np)
    """
    return np.exp(z - np.exp(z))*(1 + alpha*(np.exp(z) - 1))

In [ ]:
z = np.linspace(-10,5,100)
alphas = np.array([1, 0.43, 0.18, 0])
for alpha in alphas:
    praz = pralpha_z(z = z, alpha = alpha)
    plt.plot(z, praz)

In [ ]:
# for analytical expression of Porter Thomas distr
z = np.linspace(-8,4,1000)
colors = ['c', 'y', 'b', 'r', 'C1', 'C2']
depth = max(dfmaster['depth'].unique())
num_qubits = max(dfmaster['num_qubits'].unique())
print(num_qubits)
i = 0
plt.figure(figsize=(10,10))
rs = dfmaster['r'].unique()
rs.sort()

for r in rs:
    if r == 0.01:
        continue
    indr = dfmaster['r'].isin([r]) & dfmaster['depth'].isin([depth]) & dfmaster['num_qubits'].isin([num_qubits] )
    items = dfmaster[['puexp', 'cross_entropy_diff', 'pU', 'alpha_dig']][indr]
    meanpu = np.mean(np.array([np.array(list(tup)) for tup in items['pU']]), axis=0)
    # setting the last puexp as the measurement distribution because the number of samples drawn 
    #is not equal for each circuit drawn from the ensemble for which the sampling is done 
    lastpuexp = np.array(items['puexp'][[(i == 4) for i in range(5)]][0])
    
    #computing Np
    Np = 2**(num_qubits)*lastpuexp
    
    # computing log Np with redefinition for better numerics
    logNp = num_qubits*np.log(2) + np.log(lastpuexp)
    praz = pralpha_z(z = z, alpha = items['alpha_dig'].mean(axis=0))
    #print(items['cross_entropy_diff'])
    alpha_est = items['cross_entropy_diff'].mean(axis=0)
    alpha_dig = items['alpha_dig'].mean(axis=0)
    plt.hist(logNp, label='$\\alpha_{est} = %.2f$'%alpha_est, color = viridis(alpha_est), bins = 20, density = True, alpha=0.5 - 0.05*i)
    plt.plot(z, praz, '-', label = '$\\alpha_{dig} = %.2f}$'%alpha_dig, color = viridis(alpha_dig))#, np.exp(-logNp_ideal)
    i+=1
plt.legend()

In [ ]:
def compute_yaxis_iprk(dfmaster, k, num_qubits, depth, r):
    iprkdf = dfmaster[['r','depth']+['IPR%i'%i for i in np.arange(2,11,2)]]
    indiprk = iprkdf['r'].isin([r]) & iprkdf['depth'].isin([depth])
    #print(iprkdf[indiprk])
    #print(iprkdf[indiprk]['IPR%i'%k])
    iprk = iprkdf[indiprk]['IPR%i'%k].mean()
    # for such a large number it is more convenient to use the logarithm
    lognumber = (k-1)*np.log(2**(num_qubits)*iprk) - sum([np.log(i) for i in range(2,k+1)])
    return np.exp(lognumber)

In [ ]:
compute_yaxis_iprk(dfmaster = dfmaster, k = 2, num_qubits = max(dfmaster['num_qubits'].unique()), depth = 5, r = 0.01)

In [ ]:
def iprkplot(depth, df, ks = np.arange(2,11,2), r= 0.01):
    plt.figure()
    num_qubits = max(df['num_qubits'].unique())
    for k in ks:
        iprk = []
        ds = []
        for d in depth:
            iprk_i = compute_yaxis_iprk(dfmaster = df, k = k, num_qubits = num_qubits, depth = d, r = r)
            iprk.append(iprk_i)
            ds.append(d)
        plt.plot(ds, iprk, '-o', label = '$k = %i$'%(k)) 
        #plt.ylim(0.99,5)
        plt.yscale('log')
        plt.xlabel("Depth")
        plt.ylabel("$IPRK^{(k)} N^{k-1}/k!$")
        
    plt.legend(loc='best')

In [ ]:
iprkplot(depth = np.arange(5, 20, 1), df = dfmaster)

In [ ]:
def paperplots(depth, ce, ced, e, r, df_std):
    title = "Mean value for $r = %.3f$"%r
    plt.figure()
    plt.title(title)
    plt.plot(depth, ce, '-ok')
    plt.errorbar(depth, ce, yerr = df_std['cross_entropy'])
    plt.xlabel("Depth")
    plt.ylabel("Mean cross-entropy")
    plt.savefig('/home/IPP-HGW/joag/homework/figures_qcqi_exam/cross_entropy_r_%.3f'%r + '.png')

    plt.figure()
    plt.title(title)
    plt.plot(depth, ced, '-ok')
    plt.errorbar(depth, ced, yerr = df_std['cross_entropy_diff'])
    plt.xlabel("Depth")
    plt.ylabel("Fidelity")
    plt.savefig('/home/IPP-HGW/joag/homework/figures_qcqi_exam/cross_entropy_diff_r_%.3f'%r + '.png')
    
    
    plt.figure()
    plt.title(title)
    plt.plot(depth, e, '-ok')
    plt.plot(depth, np.ones(depth.shape)*(15*np.log(2)-1 + 0.577), '--r', label='$H(p_U))$')
    plt.errorbar(depth, e, yerr = df_std['entropy'])
    plt.xlabel("Depth")
    plt.ylabel("Mean entropy")
    plt.savefig('/home/IPP-HGW/joag/homework/figures_qcqi_exam/entropy_r_%.3f'%r + '.png')
    
    

In [ ]:
for r in sorted(dfmaster_mean_noprobs['r'].unique()):
    indr = dfmaster_mean_noprobs['r'].isin([r])
    cetest = dfmaster_mean_noprobs['cross_entropy'][indr]
    cedtest = dfmaster_mean_noprobs['cross_entropy_diff'][indr]
    etest = dfmaster_mean_noprobs['entropy'][indr]
    paperplots(depth = np.arange(5, 20, 1), ce = cetest, ced = cedtest, e = etest, r = r, df_std = dfmaster_stds[indr])

In [ ]:
def add_error(atqubit,atdepth, ondepth, onqubit):
    return (atqubit == onqubit) & (atdepth == ondepth)

def gen_rqc_single_error(num_qubits, depth, errortype = 'x', add_error = add_error, atqubit = False, atdepth = False, operands = std_operands, seed = 2022):
    reg = [i for i in range(num_qubits)]
    random.seed(seed)
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h([i for i in range(qc.num_qubits)])
    choices = [None]*num_qubits
    for i in range(depth):
        # single-qubit gate layer
        for j in range(num_qubits):
            # print(i,j,add_error(atqubit = atqubit,atdepth = atdepth, ondepth = i, onqubit =j))
            choice = random_gate(num_qubits=1, operands = operands)
            while choice == choices[j]:
                choice = random_gate(num_qubits=1, operands = operands)
            choices[j] = choice # store to make sure the same gate does not get chosen two times consequtively
            qc.unitary(choice, [j], label = choice.label)
            if add_error(atqubit = atqubit,atdepth = atdepth, ondepth = i, onqubit =j):
                if errortype == 'x':
                    qc.x(atqubit)
                if errortype == 'z':
                    qc.z(atqubit)
                
        # 2-qubit gate layer
        if i%2 == 0:
            igate = np.arange(0,num_qubits-2, 2)
            for j in igate: 
                qc.cz(j, j+1)
        else:
            igate = np.arange(1,num_qubits-1, 2)
            for j in igate:
                qc.cz(j, j+1)
        qc.barrier()
                   
    return qc


In [ ]:
qrc3 = gen_rqc_single_error(num_qubits=3, depth = 3, atdepth=2, atqubit=2, errortype='z')
qrc3.draw()

In [ ]:
from itertools import product
def fig3(num_qubits, depth, Nshots = 1, errortype = 'x'):
    
    qc_ideal = gen_rqc(num_qubits=num_qubits, depth = depth)
    qc_ideal.save_statevector()
    
    # get probabilites from ideal circuit
    
    # Create ideal simulator backend and transpile circuit
    sim_ideal = AerSimulator()
    t_qc = transpile(qc_ideal, sim_ideal)
    result_ideal = sim_ideal.run(t_qc, shots=Nshots).result()
    state_ideal = result_ideal.get_statevector()
    pideal = state_ideal.probabilities()

    perror = np.zeros((num_qubits, depth, 2**num_qubits))
    for atqubit in np.arange(0,num_qubits,1):
        for atdepth in np.arange(0,depth, 1):
            # print(atqubit, atdepth)
            qc = gen_rqc_single_error(num_qubits=num_qubits, depth = depth, errortype = errortype, atdepth=atdepth, atqubit=atqubit)
            qc.save_statevector()
            
            
            t_qc_paulierror = transpile(qc, sim_ideal)
            
            
            
            # get probabilites from circuit with the one pauli error
            result_paulierror = sim_ideal.run(t_qc_paulierror, shots=Nshots).result()
            state_error = result_paulierror.get_statevector()
            perror[atqubit,atdepth] += state_error.probabilities()
        
    # sorting
    # print(pideal.shape)
    # print(np.mean(perror, axis=0).shape)
    # print(np.mean(np.mean(perror, axis=0), axis=0))
    psorted = [pipe for pipe in zip(*sorted(zip(list(pideal), list(np.mean(np.mean(perror, axis=0), axis=0)))))]
    
    return psorted

In [ ]:
psortx = fig3(10,10)

In [ ]:
plt.title("Used X-gate for Pauli error")
plt.plot(np.arange(0,len(psortx[0]), 1), list(psortx[0]), label='No errors')
plt.plot(np.arange(0,len(psortx[0]), 1), list(psortx[1]), label='One Pauli error(averaged)')
plt.xlabel('Bit-string index $j (p(x_j)$-ordered)')
plt.ylabel('$Np$')
plt.legend(loc='best')

In [ ]:
psortz = fig3(10,10, errortype='z')

In [ ]:
plt.title("Used Z-gate for Pauli error")
plt.plot(np.arange(0,len(psortz[0]), 1), list(psortz[0]), label='No errors')
plt.plot(np.arange(0,len(psortz[0]), 1), list(psortz[1]), label='One Pauli error(averaged)')
plt.xlabel('Bit-string index $j (p(x_j)$-ordered)')
plt.ylabel('$Np$')
plt.legend(loc='best')